In [45]:
# Import packages
import requests
from datetime import datetime
from datetime import date
import time
import pandas as pd

In [50]:
class Webdriver:
    
    """
    This is a class that generates a chrome webdriver and can be used to navigate the VBL website
    """
    
    def __init__(self):
        # Automatically create a webdriver when class Webdriver is called
        self.generate_driver()
        
    def generate_driver(self):
        import pyderman as pydriver
        from selenium import webdriver
        from bs4 import BeautifulSoup
        
        try: 
            # Check if chromedriver is installed (necessary for selenium). Else, install it.
            path = pydriver.install(browser=pydriver.chrome)
            print('Installed chromedriver driver to path: %s' % path)

            # Create the chromedriver instance
            self.driver = webdriver.Chrome(path)

        except:
            print('chromedriver not installed correctly')
            
    def login(self, username, password):
        from selenium.webdriver.common.keys import Keys
        
        element_username = self.driver.find_element_by_xpath("/html/body/div[2]/form/div[1]/input")
        element_password = self.driver.find_element_by_xpath("/html/body/div[2]/form/div[2]/input")
        element_button = self.driver.find_element_by_xpath("/html/body/div[2]/form/div[4]/button")
        
        element_username.clear()
        element_username.send_keys(username)
            
        element_password.clear()
        element_password.send_keys(password)
        
        element_button.send_keys(Keys.RETURN)
        
        time.sleep(5) # make sure we are logged in before redirecting to a new page
     
    def redirect_page(self, url):   
        try:
            url = url
            self.driver.get(url)   
            time.sleep(1)
        except:
            print("Incorrect page")
        
    def export_verslag(self):
        
        # Press buton 'Verslag'
        element_button = self.driver.find_element_by_link_text("Verslag")
        element_button.click()
        time.sleep(1)
        
        # Download tables under 'Verslag'
        #elem = self.driver.find_elements_by_xpath('//*[@id="teamthuis"]/div[3]/table')
        page = BeautifulSoup(self.driver.page_source, 'html.parser')

        rows = page.select('table tbody tr')
        data = []
        for row in rows:
            tds = row.select('td')
            if len(tds) == 4:
                data.append([tds[0].text, tds[1].text, tds[2].text, tds[3].text]) # check for doubles in dict!!
        
        return data

In [51]:
class VblApi:
    
    """
    This class employs functions that use the VBL API to extract matchcodes and teamcodes
    """

    def opponents_teamguids(self, own_teamguid):
        """
        Start with our own teamguid and return a dictionary of all opponents teamnames + their guids
        """
        
        url_teamdetail = 'http://vblcb.wisseq.eu/VBLCB_WebService/data/TeamDetailByGuid?teamguid=' + own_teamguid
        opponents_teamguids = {}
        
        try:
            response = requests.get(url_teamdetail)
            for teamdetail in response.json()[0]['poules'][0]['teams']:
                opponents_teamguids[teamdetail['naam']] = teamdetail['guid'].replace(" ","+") # Teamname = key, teamguid = value. Replace spaces with '+' in key 

            opponents_teamguids = {key:val for key, val in opponents_teamguids.items() if val != own_teamguid} # Copy the dictionary but remove our own teamguid
            return(opponents_teamguids)

        except:
            print('Invalid Response or Error: ' + str(response))
        
    def opponent_matchguids(self, opponents_teamguid):
        """
        Start with a single opponents teamguid & return a list of all the matchguids that are before the current date
        """   
        url_teammatches = 'http://vblcb.wisseq.eu/VBLCB_WebService/data/TeamMatchesByGuid?teamguid=' + opponents_teamguid
        opponents_matchguids = []
        today = date.today()

        try:
            response = requests.get(url_teammatches)
            for teammatches in response.json():
                if datetime.strptime(teammatches['datumString'], "%d-%m-%Y").date() < today: # If matchday is before today, then append the matchguid to list
                    opponents_matchguids.append(teammatches['guid'])                
            return(opponents_matchguids)
        except:
            print('Invalid Response or Error: ' + str(response))

    def opponents_matches(self, opponents_teams):
        """
        Use the opponent_matchguids function to return a dictionary of all opponent names & all their matchguids before today
        """
        opponents_matches = {}

        for key, value in opponents_teams.items():
            opponents_matches[key] = self.opponent_matchguids(value)
        return(opponents_matches)

In [48]:
# read teamguids
with open("teamguids.txt") as f:
    teamguids = [line.rstrip() for line in f]

In [43]:
vbl_api = VblApi()
opponents_teamguids = vbl_api.opponents_teamguids(teamguids[1])
opponents_matchguids = vbl_api.opponents_matches(opponents_teamguids)
opponents_matchguids

{'BC Guco Lier Vzw HSE F': ['BVBL20211189OR00148407',
  'BVBL20211195OR00151823',
  'BVBL20211195OR00151824',
  'BVBL20219110ANHSE41BEG',
  'BVBL20219110ANHSE41BFG',
  'BVBL20219110ANHSE41BGA',
  'BVBL20219110ANHSE41BGD',
  'BVBL20219110ANHSE41BIG',
  'BVBL20219110BANHSE1608',
  'BVBL20219110BANHSE3208'],
 'Phantoms Basket Boom HSE E': ['BVBL20219110ANHSE41BBH',
  'BVBL20219110ANHSE41BCH',
  'BVBL20219110ANHSE41BHE',
  'BVBL20219110ANHSE41BHF',
  'BVBL20219110ANHSE41BHJ',
  'BVBL20219110BANHSE3201'],
 'BBC Schelle HSE B': ['BVBL20211216OR00163402',
  'BVBL20211216OR00163404',
  'BVBL20211216OR00163405',
  'BVBL20211216OR00163409',
  'BVBL20211216OR00163414',
  'BVBL20211216OR00163416',
  'BVBL20211216OR00163419',
  'BVBL20211216OR00163422',
  'BVBL20219110ANHSE41BCE',
  'BVBL20219110ANHSE41BEG',
  'BVBL20219110ANHSE41BEI',
  'BVBL20219110ANHSE41BFE',
  'BVBL20219110ANHSE41BHE',
  'BVBL20219110ANHSE41BKE',
  'BVBL20219110BANHSE3231',
  'BVBL20219110BANHSEVR07'],
 'Duffel K.B.B.C. HSE B'

In [52]:
# login credentials
login = pd.read_csv("login.txt", sep = ": ", header = None, engine='python')
username = login[1][0]
password = login[1][1]

### Webdriver
# Call the webdriver class to generate a webdriver instance
website = Webdriver()

# Login using login credentials
website.redirect_page("https://vblweb.wisseq.eu/Home/login") # go to login page
website.login(username, password)

# Redirect to the correct pages and extract the tables
website.redirect_page("https://vblweb.wisseq.eu/Home/MatchDetail?wedguid=BVBL20211117OR00117303&ID=Uitslag")
#website.redirect_page("https://vblweb.wisseq.eu/Home/MatchDetail?wedguid=BVBL20219110ANHSE41BAL&ID=Uitslag")

chromedriver is already installed.
Installed chromedriver driver to path: C:\Users\Lukas\Test jupyterlab\lib\chromedriver_86.0.4240.22.exe


In [ ]:
verslag = website.export_verslag()
verslag